In [1]:
!git clone https://github.com/NVlabs/stylegan.git



fatal: destination path 'stylegan' already exists and is not an empty directory.


In [2]:
cd stylegan


/kaggle/working/stylegan


In [3]:
!pip install --upgrade pip==20.2
!pip install tensorflow-gpu==1.15


Requirement already up-to-date: pip==20.2 in /opt/conda/lib/python3.7/site-packages (20.2)
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [4]:
import os
import pickle
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import config


In [5]:
# part 1a
tflib.init_tf()
_G, _D, Gs = pickle.load(open('/kaggle/input/saransh/karras2019stylegan-ffhq-1024x1024.pkl','rb'))
Gs.print_layers()

import tensorflow as tf



2023-04-25 13:46:10.521887: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcudart.so.10.0'; dlerror: libcudart.so.10.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/opt/conda/lib
2023-04-25 13:46:10.522103: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcublas.so.10.0'; dlerror: libcublas.so.10.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/opt/conda/lib
2023-04-25 13:46:10.522275: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcufft.so.10.0';


Gs                              Params    OutputShape          WeightShape     
---                             ---       ---                  ---             
latents_in                      -         (?, 512)             -               
labels_in                       -         (?, 0)               -               
lod                             -         ()                   -               
dlatent_avg                     -         (512,)               -               
G_mapping/latents_in            -         (?, 512)             -               
G_mapping/labels_in             -         (?, 0)               -               
G_mapping/PixelNorm             -         (?, 512)             -               
G_mapping/Dense0                262656    (?, 512)             (512, 512)      
G_mapping/Dense1                262656    (?, 512)             (512, 512)      
G_mapping/Dense2                262656    (?, 512)             (512, 512)      
G_mapping/Dense3                262656 

In [11]:

Gs.list_layers()

with tf.Session() as sess:
   
    sess.run(tf.global_variables_initializer())
    
    layer_weights = tf.get_default_graph().get_tensor_by_name('G_synthesis/4x4/Conv/weight:0')
    biases = tf.get_default_graph().get_tensor_by_name('G_synthesis/4x4/Conv/bias:0')
    print(layer_weights.eval())
    print(biases.eval())


[[[[-0.45683435 -0.20399195 -1.206372   ...  1.0364239  -0.07009938
     1.0424279 ]
   [ 1.107865    2.3977664   1.8124753  ... -0.01523588  0.19548818
     0.9866782 ]
   [ 1.10147    -1.4211909   1.987521   ...  0.1169506  -0.23867512
    -0.38604587]
   ...
   [ 0.9012106   0.5203432   0.49768603 ... -0.5574434   0.26803485
     0.05622844]
   [ 1.5229168   0.6720213  -0.47867265 ...  0.7455013   0.18428905
     0.00866753]
   [-2.0651069  -2.6019592   0.82074535 ...  0.4180719  -0.0283604
     2.1503925 ]]

  [[-0.9495489  -0.6798598   1.1403413  ...  0.67835206 -2.177369
     2.2136106 ]
   [ 0.6304233   0.05862745 -0.89062184 ... -0.58661425  1.2595974
     0.3388388 ]
   [ 0.22916374  0.1887568  -1.4562565  ... -0.44717994 -1.9848218
     0.3633821 ]
   ...
   [-0.4778629   0.5912875   0.668814   ... -0.7226262   1.10232
     0.29148757]
   [-1.356259    0.1401832   0.664807   ... -0.6409763  -0.26875156
    -0.03437966]
   [ 0.47889152 -0.4195626  -1.3002104  ...  1.5543226   

In [ ]:
# part 1b
layer = 8  

# Generate two random latent vectors
latent_size = Gs.input_shape[1]
latents1 = np.random.randn(1, latent_size)
latents2 = np.random.randn(1, latent_size)

# Generate images from both latent vectors
with tf.device('/cpu:0'):
    images1 = Gs.run(latents1, None, truncation_psi=0.7, randomize_noise=True, output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True))
    images2 = Gs.run(latents2, None, truncation_psi=0.7, randomize_noise=True, output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True))

# Perform style mixing between the two latent vectors
dlatent1 = Gs.components.mapping.run(latents1, None)
dlatent2 = Gs.components.mapping.run(latents2, None)
dlatent_mix = np.concatenate((dlatent1[:, :layer], dlatent2[:, layer:]), axis=1)

with tf.device('/cpu:0'):
    images_mix = Gs.components.synthesis.run(dlatent_mix, randomize_noise=True, output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True))

# Display the generated images
from PIL import Image
Image.fromarray(images1[0], 'RGB').show()
Image.fromarray(images2[0], 'RGB').show()
Image.fromarray(images_mix[0], 'RGB').show()

In [14]:
#part 1 c)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    layer_weights = tf.get_default_graph().get_tensor_by_name('G_synthesis_1/ToRGB_lod8/weight:0')
    print(layer_weights.eval())

[[[[-0.53808254 -0.4457194   0.8153907 ]
   [-0.81344473  0.9510818   0.41592908]
   [-0.8689108   0.05683148  0.47085947]
   ...
   [-2.1535792   0.7405902  -1.3714038 ]
   [-1.1777053   1.1387714  -0.49882436]
   [ 0.43906003 -1.6398451   0.13521947]]]]


In [ ]:
# part 1f
# for three latents
layer = 8  

# Generate two random latent vectors
latent_size = Gs.input_shape[1]
latents1 = np.random.randn(1, latent_size)
latents2 = np.random.randn(1, latent_size)
latents3 = np.random.randn(1, latent_size)

# Generate images from both latent vectors
with tf.device('/cpu:0'):
    images1 = Gs.run(latents1, None, truncation_psi=0.7, randomize_noise=True, output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True))
    images2 = Gs.run(latents2, None, truncation_psi=0.7, randomize_noise=True, output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True))
    images3 = Gs.run(latents3, None, truncation_psi=0.7, randomize_noise=True, output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True))

# Perform style mixing between the two latent vectors
dlatent1 = Gs.components.mapping.run(latents1, None)
dlatent2 = Gs.components.mapping.run(latents2, None)
dlatent3 = Gs.components.mapping.run(latents3, None)
dlatent_mix = np.concatenate((dlatent1[:, :layer], dlatent2[:, layer:], dlatent3[:, layer:]), axis=1)

with tf.device('/cpu:0'):
    images_mix = Gs.components.synthesis.run(dlatent_mix, randomize_noise=True, output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True))

# Display the generated images
from PIL import Image
Image.fromarray(images1[0], 'RGB').show()
Image.fromarray(images2[0], 'RGB').show()
Image.fromarray(images3[0], 'RGB').show()
Image.fromarray(images_mix[0], 'RGB').show()